In [3]:
from pyspark import SparkConf, SparkContext
conf = SparkConf().setMaster("local").setAppName("restaurant-review-average")
spark = SparkContext(conf = conf).getOrCreate()

In [2]:
spark.stop() #그냥 오류나면 눌러보기~

In [4]:
import os

In [5]:
directory = os.path.join(os.getcwd(), "data") #현재 위치를 불러와 그 밑에 data폴더를 추가
filename = "restaurant_reviews.csv"
filepath = os.path.join(directory, filename)

In [6]:
# RDD 생성
lines = spark.textFile("file:///" + filepath.replace("\\", "/"))
lines

file:////home/lab02/src/data/restaurant_reviews.csv MapPartitionsRDD[1] at textFile at NativeMethodAccessorImpl.java:0

In [11]:
lines.take(10)

['id,item,cateogry,reviews,',
 '0,짜장면,중식,125,',
 '1,짬뽕,중식,235,',
 '2,김밥,분식,32,',
 '3,떡볶이,분식,534,',
 '4,라멘,일식,223,',
 '5,돈가스,일식,52,',
 '6,우동,일식,12,',
 '7,쌀국수,아시안,312,',
 '8,햄버거,패스트푸드,12,']

In [12]:
header = lines.first() #1번째 줄(컬럼명)만 출력
header

'id,item,cateogry,reviews,'

In [41]:
datas = lines.filter(lambda row: row != header) #컬럼명인 1번째 줄을 제외하고 출력
datas.take(9)

['0,짜장면,중식,125,',
 '1,짬뽕,중식,235,',
 '2,김밥,분식,32,',
 '3,떡볶이,분식,534,',
 '4,라멘,일식,223,',
 '5,돈가스,일식,52,',
 '6,우동,일식,12,',
 '7,쌀국수,아시안,312,',
 '8,햄버거,패스트푸드,12,']

In [36]:
# 함수로 따로 만들어서 진행!
def parse(row):
    fields = row.split(",")
    food = fields[1]
    #category = fields[2]
    reviews = int(fields[3])
    
    return food, reviews

In [37]:
parse('0,짜장면,중식,125,')

('짜장면', 125)

In [38]:
category_reviews = datas.map(parse)

In [39]:
category_reviews.take(5)

[('짜장면', 125), ('짬뽕', 235), ('김밥', 32), ('떡볶이', 534), ('라멘', 223)]

In [40]:
category_review_count = category_reviews.mapValues(lambda x : (x,1))
category_review_count.take(3) #데이터 형태가 변한 것을 확인

[('짜장면', (125, 1)), ('짬뽕', (235, 1)), ('김밥', (32, 1))]

## 공식 `(125,1)`

- 첫번째 공식 `(125,1)` → `x`
- 두번째 공식 `(235,1)` → `y`

### 리뷰의 갯수의 합
`x[0] + y[0]`

### 건수의 합
`x[1] + y[1]`

In [42]:
# 카테고리별 합계
reduce_rdd = category_review_count.reduceByKey(lambda x,y : (x[0]+y[0], x[1]+y[1]))
reduce_rdd.take(3)

[('짜장면', (125, 1)), ('짬뽕', (235, 1)), ('김밥', (32, 1))]

In [43]:
reduce_rdd.collect()

[('짜장면', (125, 1)),
 ('짬뽕', (235, 1)),
 ('김밥', (32, 1)),
 ('떡볶이', (534, 1)),
 ('라멘', (223, 1)),
 ('돈가스', (52, 1)),
 ('우동', (12, 1)),
 ('쌀국수', (312, 1)),
 ('햄버거', (12, 1)),
 ('치킨', (23, 1))]

In [44]:
# 평균
average = reduce_rdd.mapValues(lambda x: x[0]/x[1])
average.collect()

[('짜장면', 125.0),
 ('짬뽕', 235.0),
 ('김밥', 32.0),
 ('떡볶이', 534.0),
 ('라멘', 223.0),
 ('돈가스', 52.0),
 ('우동', 12.0),
 ('쌀국수', 312.0),
 ('햄버거', 12.0),
 ('치킨', 23.0)]

In [45]:
spark.stop() #마지막에 꼭 실행주어야함